# Section 1

### download data

In [1]:
import requests
from bs4 import BeautifulSoup
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

### transform

In [2]:
import pandas as pd

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

# Section 2

In [3]:
#%pip install wget

In [4]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
Geospatial_Coordinates = pd.read_csv(url)

In [5]:
data = pd.merge(df, Geospatial_Coordinates, left_on='PostalCode', right_on='Postal Code', how='left').drop('Postal Code', axis=1)
data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Section 3

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#%pip install folium
import folium # map rendering library

#%pip install geopy
from geopy.geocoders import Nominatim

print('Libraries imported.')

Libraries imported.


In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
neighborhoods = data
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [9]:
CLIENT_ID = 'HK3CTO3E2IPWHPOYBXCZL12C3HIO53SKOSJOF0EMZNVQPFXU' # your Foursquare ID
CLIENT_SECRET = 'EAXJW0NG5H1LWNOHBKSG3K3J0NYDO3O4HRHHYEVN5L04B5LU' # your Foursquare Secret
ACCESS_TOKEN = '3HF5FFVZC42ODXNBZYJDXKZSTBCH3OMRG1JOZ25OOAWYLZDH'
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
# looks at what are the nearby venues
toronto_nearby = getNearbyVenues(neighborhoods['Neighborhood'],neighborhoods['Latitude'], neighborhoods['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [12]:
# one hot
toronto_venues = toronto_nearby
# Lets analyze the dataframes
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


# Lets check the onehot dataframe
toronto_onehot.tail()


,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
2120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [13]:
#let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#print each neighborhood along with the top 5 most common venues
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

                       venue  freq
0             Clothing Store   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2      Playground  0.14
3  Sandwich Place  0.14
4             Pub  0.14


                 venue  freq
0          Coffee Shop  0.10
1                 Bank  0.10
2          Pizza Place  0.05
3           Restaurant  0.05
4  Fried Chicken Joint  0.05


                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4  Moroccan Restaurant  0.00


                  venue  freq
0    Italian Restaurant  0.09
1        Sandwich Place  0.09
2           Coffee Shop  0.09
3     Indian Restaurant  0.04
4  Fast Food Restaurant  0.04


                venue  freq
0              Bakery  0.05
1        Cocktail Bar  0.05
2         Coffee Shop  0

                  venue  freq
0        Massage Studio  0.17
1    Falafel Restaurant  0.17
2  Caribbean Restaurant  0.17
3           Coffee Shop  0.17
4         Metro Station  0.17


                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


         venue  freq
0  Coffee Shop  0.18
1         Café  0.06
2        Diner  0.06
3  Yoga Studio  0.03
4   Restaurant  0.03


                         venue  freq
0               Breakfast Spot  0.14
1                    Gift Shop  0.14
2                  Coffee Shop  0.07
3  Eastern European Restaurant  0.07
4                          Bar  0.07


                venue  freq
0         Pizza Place  0.17
1  Athletics & Sports  0.08
2            Pharmacy  0.08
3         Flea Market  0.08
4                Café  0.08


                        venue  freq
0        Fas

In [14]:
#lets sort the venues in descending order and display the top 10 venues for each neighborhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
	
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))




In [15]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
#Lets preview our sorted venues dataframe
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Playground,Sandwich Place,Pub,Coffee Shop,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Restaurant,Fried Chicken Joint,Bridal Shop,Mobile Phone Shop,Gas Station,Intersection,Diner
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Indian Restaurant,Fast Food Restaurant,Butcher,Liquor Store,Café,Restaurant,Juice Bar


In [16]:
#Run k-means to cluster the neighborhood into 7 clusters.

# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])

[6 0 6 6 6 6 6 6 6 4]


In [17]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
toronto_data = neighborhoods

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.dropna() #dropping the null values in columns as they have little/no effect on our cluster

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Fast Food Restaurant,Park,Construction & Landscaping,Food & Drink Shop,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,6.0,Intersection,French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6.0,Coffee Shop,Park,Pub,Bakery,Theater,Café,Mexican Restaurant,Beer Store,French Restaurant,Breakfast Spot
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6.0,Clothing Store,Furniture / Home Store,Boutique,Vietnamese Restaurant,Miscellaneous Shop,Sporting Goods Shop,Coffee Shop,Accessories Store,Medical Center,Massage Studio
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,6.0,Coffee Shop,Café,Diner,Yoga Studio,Restaurant,College Auditorium,Bank,Sushi Restaurant,Beer Bar,Fried Chicken Joint


In [18]:
#Now lets visualize the clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [87]:
# get potential places with top 3 for Restaurant
potential_places = toronto_merged.loc[toronto_merged['1st Most Common Venue'].str.startswith('Restaurant') | toronto_merged['2nd Most Common Venue'].str.startswith('Restaurant') | toronto_merged['3rd Most Common Venue'].str.startswith('Restaurant')]
potential_places

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M3C,North York,Don Mills South,43.725900,-79.340923,6.0,Restaurant,Coffee Shop,Gym,Clothing Store,Art Gallery,Supermarket,Beer Store,Italian Restaurant,Sandwich Place,Bike Shop
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6.0,Coffee Shop,Café,Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Beer Bar,Park,Gastropub
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,6.0,Men's Store,Bar,Restaurant,Café,Vietnamese Restaurant,Asian Restaurant,Juice Bar,Korean Restaurant,Beer Store,Bistro
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,6.0,Coffee Shop,Restaurant,Café,Hotel,Gym,Vegetarian / Vegan Restaurant,Deli / Bodega,Japanese Restaurant,Bakery,American Restaurant
56,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,6.0,Restaurant,Convenience Store,Sandwich Place,Discount Store,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
59,M2N,North York,Willowdale South,43.770120,-79.408493,6.0,Café,Ramen Restaurant,Restaurant,Pizza Place,Sushi Restaurant,Shopping Mall,Coffee Shop,Bank,Liquor Store,Lounge
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,6.0,Coffee Shop,Restaurant,Light Rail Station,Sushi Restaurant,Fried Chicken Joint,Supermarket,Café,Pub,Bank,American Restaurant
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,6.0,Coffee Shop,Restaurant,Bakery,Café,Pub,Pizza Place,Italian Restaurant,Bank,General Entertainment,Butcher
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,5.0,Pizza Place,Restaurant,Brewery,Skate Park,Burrito Place,Light Rail Station,Butcher,Garden,Farmers Market,Garden Center


In [88]:

#Now lets visualize the neighbourhood for shotlist
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(potential_places['Latitude'], potential_places['Longitude'], potential_places['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
#download demographic data from https://open.toronto.ca/dataset/neighbourhood-profiles/

population = pd.read_csv('neighbourhood-profiles-2016.csv')

potential_places = potential_places.join(population.set_index('Neighborhood'), on='Neighborhood')
potential_places = potential_places.dropna() 
potential_places

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,density
13,M3C,North York,Don Mills South,43.725900,-79.340923,6.0,Restaurant,Coffee Shop,Gym,Clothing Store,Art Gallery,Supermarket,Beer Store,Italian Restaurant,Sandwich Place,Bike Shop,"2,775"
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6.0,Coffee Shop,Café,Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Beer Bar,Park,Gastropub,"44,321"
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,6.0,Men's Store,Bar,Restaurant,Café,Vietnamese Restaurant,Asian Restaurant,Juice Bar,Korean Restaurant,Beer Store,Bistro,"12,859"
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,6.0,Coffee Shop,Restaurant,Café,Hotel,Gym,Vegetarian / Vegan Restaurant,Deli / Bodega,Japanese Restaurant,Bakery,American Restaurant,"3,710"
56,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,6.0,Restaurant,Convenience Store,Sandwich Place,Discount Store,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,"6,442"
59,M2N,North York,Willowdale South,43.770120,-79.408493,6.0,Café,Ramen Restaurant,Restaurant,Pizza Place,Sushi Restaurant,Shopping Mall,Coffee Shop,Bank,Liquor Store,Lounge,"10,087"
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,6.0,Coffee Shop,Restaurant,Light Rail Station,Sushi Restaurant,Fried Chicken Joint,Supermarket,Café,Pub,Bank,American Restaurant,"4,380"
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,6.0,Coffee Shop,Restaurant,Bakery,Café,Pub,Pizza Place,Italian Restaurant,Bank,General Entertainment,Butcher,"8,335"


In [90]:
potential_places['density_int'] = pd.to_numeric(potential_places['density'].str.replace(',',''))# = int(potential_places['density'])
final_list = potential_places[['Neighborhood','density_int']].sort_values(by = 'density_int', ascending=False, inplace=False)
final_list.head(1)

,Neighborhood,density_int
15,St. James Town,44321


In [91]:
final_list = potential_places[1:2]

In [92]:
#Now lets visualize the neighbourhood for final winner
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_list['Latitude'], final_list['Longitude'], final_list['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters